In [ ]:
## json to table
# json은 {키:값} 형식으로 제공하는 파일

# 1. json형식의 데이터를 디코딩
# 2. 데이터프레임 형식으로 변경하면 테이블 구조의 컬럼 단위로 저장

In [ ]:
import pandas as pd
import pymysql
import json

config = {
    'host' : '127.0.0.1',       # localhost 주소
    'user'  : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True
}

# labels = pd.read_json('data/labels.json')
file = open('data/labels.json', 'r', encoding='utf8')

# 디코딩 : json -> object
lines = json.load(file)         # => list 내의 dict 형태로 파일이 읽어짐
# type(lines)      # <class 'list'>
# len(lines)       # 30
# type(lines[0])   # <class 'dict'>

## DataFrame 생성
df = pd.DataFrame(lines)
# df.info()  # 테이블 컬럼 자료형 선언을 위해 저장된 데이터의 자료형 조회하기
# df.head()  # head() : 상위 5개 출력 / head(10) : 상위 10개 출력

try :
    conn = pymysql.Connect(**config)
    cursor = conn.cursor()
    
    sql = """CREATE TABLE IF NOT EXISTS labels (
            id int not null, url varchar(150) not null, 
            name varchar(50) not null, 
            color varchar(10) not null, def char(5)
            )"""
    cursor.execute(sql)
    
    # 레코드 조회 후 있는 경우 출력, 없는 경우 추가
    cursor.execute("SELECT * FROM labels")
    rows = cursor.fetchall()
    
    if rows :
        print("테이블 레코드 출력")
        for row in rows :
            print(row)
        print("전체 레코드 수 :",len(rows))
    else :
        for line in df.values :
            sql = f"INSERT INTO labels VALUES ({line[0]},'{line[1]}','{line[2]}','{line[3]}','{line[4]}')"            
            # sql = "INSERT INTO labels VALUES ({df.id[i]},'{df.url[i]}','{df.name[i]}','{df.color[i]}','{df.def[i]}')"
            cursor.execute(sql)
        conn.commit()

except Exception as e :
    print('DB연동 에러 발생 !',e)
    conn.rollback()     # INSERT, UPDATE, DELETE 시 사용
    
finally :
    cursor.close()
    conn.close()

In [64]:
# 예제 - emp.csv 파일을 처리하여 JSON data로 변환 후 저장하기
# 저장된 json 파일을 불러와서 DB 작업
import pandas as pd
import json

emp = pd.read_csv('data/emp.csv')
total = []      # 모든 딕셔너리 쌍을 담을 리스트
for i in range(len(emp)) :
    item = {}
    for col in emp.columns :
        item.update({col: str(emp[col][i])})    # json 변환 시 자료형이 numpy.int64가 되므로 int/str로 바꿔준다
    total.append(item) 
    
with open('data/emp.json','w',encoding='utf-8') as f : # JSON 데이터로 변환에서 쓰기       
    jstr = json.dumps(total,ensure_ascii=False, indent=4)
    f.write(jstr)

data = open('data/emp.json','r',encoding='utf8')
data = json.load(data)
df = pd.DataFrame(data)
no = df['No']; name = df['Name']; pay = df['Pay']

config = {
'host' : '127.0.0.1',       # localhost 주소
'user'  : 'scott',
'password' : 'tiger',
'database' : 'work',
'port' : 3306,
'charset' : 'utf8',
'use_unicode' : True
}

try :   
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS emp (no int not null, name varchar(10) not null, pay int not null)"
    cursor.execute(sql)
    
    sql = "SELECT * FROM emp"
    cursor.execute(sql)
    rows = cursor.fetchall()
    if not rows :
        for i in range(len(df)) :
            sql = f"INSERT INTO emp VALUES ({no[i]},'{name[i]}',{pay[i]})"
            cursor.execute(sql)
        conn.commit()
    else :
        for row in rows :
            print(row)      
    

except Exception as e :
    print('오류 발생 !',e)
    conn.rollback()
finally : 
    conn.close()
    cursor.close()




(101, '홍길동', 150)
(102, '이순신', 450)
(103, '강감찬', 500)
(104, '유관순', 350)
(105, '김유신', 400)
